In [36]:
String userHomeDir = System.getProperty("user.home");
String localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";

In [37]:
%%bash 
rm -rf \{userHomeDir}/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/dev/ai4j/openai4j

In [ ]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
%dependency /add dev.ai4j:openai4j:0.24-SNAPSHOT
%dependency /resolve

In [ ]:
%load ./hello-world.ipynb

In [ ]:
import dev.ai4j.openai4j.OpenAiClient;
import dev.ai4j.openai4j.chat.ChatCompletionRequest;
import dev.ai4j.openai4j.chat.ChatCompletionResponse;

OpenAiClient client = OpenAiClient.builder()
        .openAiApiKey(OPENAI_API_KEY)
        .logRequests()
        .logResponses()
        .build();

ChatCompletionRequest request = ChatCompletionRequest.builder()
        .model("gpt-4o-mini")
        .addUserMessage("こんにちは")
        .build();

ChatCompletionResponse response = client.chatCompletion(request).execute();

String textResponse = response.content();
System.out.println("テキスト応答: " + textResponse);


## Audio Generation
https://platform.openai.com/docs/guides/audio のJava実装
openai4jをオーディオ対応にカスタマイズ

In [ ]:
import dev.ai4j.openai4j.OpenAiClient;
import dev.ai4j.openai4j.chat.ChatCompletionRequest;
import dev.ai4j.openai4j.chat.ChatCompletionResponse;
import dev.ai4j.openai4j.chat.AssistantMessage;

import javax.sound.sampled.*;
import java.io.ByteArrayInputStream;
import java.io.FileOutputStream;
import java.io.IOException;
import java.util.Arrays;
import java.util.Base64;
import static java.util.Arrays.asList;

// Original Code
import dev.ai4j.openai4j.chat.AudioSettings;

OpenAiClient client = OpenAiClient.builder()
        .openAiApiKey(OPENAI_API_KEY)
        .logRequests()   // リクエストをログに出力
        .logResponses()  // レスポンスをログに出力
        .build();

var audioSettings = new AudioSettings("nova", "wav");

ChatCompletionRequest request = ChatCompletionRequest.builder()
        .model("gpt-4o-audio-preview")
        .addUserMessage("こんにちは！今JJUG(ジェイジャグ)の会場にいます。LangChain4jのセッションをします。会場にいる人に短めのご挨拶を")
        .audioSettings(audioSettings)
        .modalities(asList("text", "audio"))
        .build();

ChatCompletionResponse response = client.chatCompletion(request).execute();
// Original Code
AssistantMessage.AudioData audioData = response.choices().get(0).message().audio();
String data = audioData.data();

byte[] audioBytes = Base64.getDecoder().decode(data);
var bais = new ByteArrayInputStream(audioBytes);
var audioInputStream = AudioSystem.getAudioInputStream(bais);
var audioFormat = audioInputStream.getFormat();
var info = new DataLine.Info(Clip.class, audioFormat);
var clip = (Clip) AudioSystem.getLine(info);
clip.open(audioInputStream);
clip.start();
while (clip.isRunning()) {
    Thread.sleep(100);
}
clip.close();

audioData.transcript()